In [1]:
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
from scipy.linalg import sqrtm
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster

# Load Data

In [6]:
project_folder = os.path.abspath('..')

# === Load & parse data ===
fitted_params = pd.read_csv(f'{project_folder}/results/gmm_results_1900thresh_diffscatter/gmm_fits_45g_1000s.csv')
fitted_params['period'] = fitted_params['filename'].str.extract(r'_(\d+)p_')[0].astype(int)
fitted_params['scan'] = fitted_params['filename'].str.extract(r'p_(\d+)\.cbf')[0].astype(int)

image_folder = os.path.join(project_folder, 'diffraction_images')
files = sorted([f for f in os.listdir(image_folder) if f.endswith('.cbf')])
roi_df = pd.read_csv(f'{project_folder}/results/ROI/ROI_800thresh.csv')

# Clustering Functions

In [7]:
def reorder_clusters(group, labels):
    """Assign clusters based on lowest mean_y first, then mean_x."""
    cluster_means = pd.DataFrame({
        'cluster': np.unique(labels),
        'mean_x': [group.loc[labels == c, 'mean_x'].mean() for c in np.unique(labels)],
        'mean_y': [group.loc[labels == c, 'mean_y'].mean() for c in np.unique(labels)]
    })
    
    # Step 1: Find the cluster with the lowest mean_y (Cluster 1)
    cluster_means = cluster_means.sort_values(by='mean_y')
    cluster1 = cluster_means.iloc[0]  # First row is the lowest y
    cluster1_label = cluster1['cluster']

    # Step 2: Remove Cluster 1 and sort remaining clusters by mean_x
    remaining_clusters = cluster_means[cluster_means['cluster'] != cluster1_label]
    remaining_clusters = remaining_clusters.sort_values(by='mean_x', ascending=False).reset_index(drop=True)

    # Step 3: Assign cluster indices
    new_labels = {cluster1_label: 1}
    for idx, row in enumerate(remaining_clusters.itertuples(), start=2):
        new_labels[row.cluster] = idx

    # Step 4: Apply the new cluster indices
    return np.array([new_labels[label] for label in labels])

# K-Means

In [8]:
# === Cluster assignment by scan ===
for scan, group in tqdm(fitted_params.groupby('filename'), desc='Clustering data:'):
    mu_x = group['mean_x'].values
    mu_y = group['mean_y'].values
    data = list(zip(mu_x, mu_y))

    n_clusters = min(3, len(data))
    kmeans = KMeans(n_clusters=n_clusters, init="k-means++")
    kmeans.fit(data)

    labels = kmeans.predict(data)
    sorted_labels = reorder_clusters(group, labels)
    fitted_params.loc[group.index, 'cluster'] = sorted_labels

Clustering data:: 100%|██████████| 16502/16502 [00:57<00:00, 287.50it/s]


In [ ]:
fitted_params.to_csv(f'{project_folder}/results/cluster_results/kmeans_plusinit.csv', index=False)

## Kmeans Wasserstein Distance

In [ ]:
def sqrtm_psd(mat):
    """Compute the square root of a 2x2 positive semi-definite matrix."""
    eigvals, eigvecs = np.linalg.eigh(mat)
    sqrt_eigvals = np.sqrt(np.maximum(eigvals, 0))
    return eigvecs @ np.diag(sqrt_eigvals) @ eigvecs.T

def gaussian_w2(mu1, cov1, mu2, cov2):
    """2-Wasserstein distance between two 2D Gaussians."""
    mean_term = np.sum((mu1 - mu2) ** 2)
    sqrt_cov2 = sqrtm_psd(cov2)
    cov_product = sqrtm_psd(sqrt_cov2 @ cov1 @ sqrt_cov2)
    cov_term = np.trace(cov1 + cov2 - 2 * cov_product)
    return np.sqrt(mean_term + cov_term)

def pairwise_gaussian_w2(means, covs):
    """Compute all pairwise 2-Wasserstein distances between Gaussians."""
    n = len(means)
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = gaussian_w2(means[i], covs[i], means[j], covs[j])
            D[i, j] = D[j, i] = d
    return D

def build_cov_matrix(sx, sy, rho):
    """Convert std devs and correlation into covariance matrix."""
    return np.array([
        [sx**2, rho * sx * sy],
        [rho * sx * sy, sy**2]
    ])

In [18]:
# === Clustering by scan ===
for scan, group in tqdm(fitted_params.groupby('filename'), desc='Clustering Gaussians'):
    mu = group[['mean_x', 'mean_y']].values
    sigmas = group[['sigma_x', 'sigma_y']].values
    rhos = group['rho'].values

    covs = [build_cov_matrix(sx, sy, rho) for (sx, sy), rho in zip(sigmas, rhos)]
    
    D = pairwise_gaussian_w2(mu, covs)

    n_clusters = min(3, len(mu))
    kmedoids = KMedoids(n_clusters=n_clusters, metric='precomputed', init='k-medoids++', random_state=0)
    kmedoids.fit(D)

    labels = kmedoids.labels_
    sorted_labels = reorder_clusters(group, labels)  # optional: normalize label order
    fitted_params.loc[group.index, 'cluster'] = sorted_labels

Clustering Gaussians: 100%|██████████| 16502/16502 [12:03<00:00, 22.82it/s]


In [ ]:
fitted_params.to_csv(f'{project_folder}/results/cluster_results/kmedoid_wasserstein.csv', index=False)

# Hierarchical Clustering

In [8]:
def get_gaussian_params(row):
    mu = np.array([row['mean_x'], row['mean_y']])
    sigma_x = row['sigma_x']
    sigma_y = row['sigma_y']
    rho = row['rho']
    cov = np.array([[sigma_x**2, rho * sigma_x * sigma_y],
                    [rho * sigma_x * sigma_y, sigma_y**2]])
    return mu, cov

def wasserstein_2(mu1, cov1, mu2, cov2):
    diff = mu1 - mu2
    sqrt_cov1 = sqrtm(cov1)
    inner = sqrtm(sqrt_cov1 @ cov2 @ sqrt_cov1)
    if np.iscomplexobj(inner):
        inner = inner.real  # drop imaginary noise
    return np.linalg.norm(diff)**2 + np.trace(cov1 + cov2 - 2 * inner)

In [9]:
for filename, group in tqdm(fitted_params.groupby('filename'), desc='Clustering data:'):
    # Filter the row corresponding to the current filename
    roi_row = roi_df[roi_df['filename'] == filename]

    params_df = fitted_params[fitted_params['filename']==filename].copy()

    n = len(params_df)
    dist_matrix = np.zeros((n, n))

    gaussians = [get_gaussian_params(row) for _, row in params_df.iterrows()]

    for i in range(n):
        for j in range(i + 1, n):
            mu1, cov1 = gaussians[i]
            mu2, cov2 = gaussians[j]
            dist = wasserstein_2(mu1, cov1, mu2, cov2)
            dist_matrix[i, j] = dist_matrix[j, i] = dist

    # Convert to condensed form
    condensed_dist = squareform(dist_matrix)

    # Linkage (use 'average', 'complete', 'ward' etc.)
    Z = linkage(condensed_dist, method='ward')

    # Get flat clusters (e.g., force 3 clusters)
    cluster_labels = fcluster(Z, t=3, criterion='maxclust')
    sorted_labels = reorder_clusters(group, cluster_labels)
    fitted_params.loc[group.index, 'cluster'] = sorted_labels

Clustering data:: 100%|██████████| 16502/16502 [1:19:38<00:00,  3.45it/s]


In [11]:
fitted_params.to_csv(f'{project_folder}/results/cluster_results/hierarchical_wasserstein_wardlinkage.csv')

# Calculate Global Params

## Functions

In [12]:
def compute_combined_gaussian_params(df):
    df = df.reset_index(drop=True)

    A = df['amplitude'].to_numpy()
    weights = A / np.sum(A)
    mu_x = np.sum(weights * df['mean_x'].to_numpy())
    mu_y = np.sum(weights * df['mean_y'].to_numpy())

    total_intensity = 0
    Sigma = np.zeros((2, 2))

    for i, row in df.iterrows():
        sigma_x_i, sigma_y_i, rho_i = row['sigma_x'], row['sigma_y'], row['rho']
        mu_x_i, mu_y_i = row['mean_x'], row['mean_y']
        Sigma_i = np.array([
            [sigma_x_i**2, rho_i * sigma_x_i * sigma_y_i],
            [rho_i * sigma_x_i * sigma_y_i, sigma_y_i**2]
        ])

        det_Sigma_i = np.linalg.det(Sigma_i)
        intensity_i = row['amplitude'] * np.sqrt((2 * np.pi)**2 * det_Sigma_i)
        
        # Add to the total intensity
        total_intensity += intensity_i

        mean_diff = np.array([[mu_x_i - mu_x], [mu_y_i - mu_y]])
        Sigma += weights[i] * (Sigma_i + mean_diff @ mean_diff.T)

    sigma_x_total = np.sqrt(Sigma[0, 0])
    sigma_y_total = np.sqrt(Sigma[1, 1])
    rho_total = Sigma[0, 1] / (sigma_x_total * sigma_y_total)

    return total_intensity, mu_x, mu_y, sigma_x_total, sigma_y_total, rho_total


def compute_gaussian_rotation(sigma_x, sigma_y, rho):
    # Covariance matrixlinal
    Sigma = np.array([[sigma_x**2, rho * sigma_x * sigma_y], 
                      [rho * sigma_x * sigma_y, sigma_y**2]])

    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(Sigma)

    # Sort eigenvalues and eigenvectors (largest eigenvalue first)
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Compute sigma_major and sigma_minor
    sigma_major = np.sqrt(eigenvalues[0])
    sigma_minor = np.sqrt(eigenvalues[1])

    # Compute rotation angle phi
    phi = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])

    return sigma_major, sigma_minor, phi

## Load Data

In [ ]:
#fitted_params=pd.read_csv('final_results_clustering/kmeans_plusinit.csv')

In [13]:
results = []

for _, group in tqdm(fitted_params.groupby('filename')):
    for cluster in group['cluster'].unique():
        cluster_group = group[group['cluster'] == cluster]

        if cluster_group.empty:
            continue

        try:
            intensity, mu_x, mu_y, sigma_x, sigma_y, rho = compute_combined_gaussian_params(cluster_group)
            sigma_major, sigma_minor, phi = compute_gaussian_rotation(sigma_x, sigma_y, rho)
        except Exception as e:
            print(f"Skipping cluster {cluster} in {group['filename'].iloc[0]} due to error: {e}")
            continue

        results.append({
            'filename': cluster_group['filename'].iloc[0],
            'cluster': cluster,
            'intensity': intensity,
            'mean_x': mu_x,
            'mean_y': mu_y,
            'sigma_x': sigma_x,
            'sigma_y': sigma_y,
            'rho': rho,
            'fwhm_major': sigma_major * 2.3548,
            'fwhm_minor': sigma_minor * 2.3548,
            'phi': phi,
        })

all_results_df = pd.DataFrame(results)

100%|██████████| 16502/16502 [02:15<00:00, 122.16it/s]


In [14]:
all_results_df.to_csv(f'{project_folder}/results/cluster_results/spot_features/hierarchical_wasserstein_wardlinkage_features.csv')